# Group Proposal DSCI100

#### Kole Laviolette 72088131 Jerry Qi73382533
#### everyone put ur name here

# Introduction 

text

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [3]:
read_csv("https://raw.githubusercontent.com/JerryQi77/DSCI_Group-Project/main/DSCI_Group_project?token=GHSAT0AAAAAABZODHTLLIBEQEOCON2SJKP6Y2ZYXBA")

Rows: 250 Columns: 30
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): TEAM, First Player, Second Player, Third Player, Fourth Player, Fi...
dbl (24): ...1, GP, MIN, PTS, FGM, FGA, FG%, 3PM, 3PA, 3P%, FTM, FTA, FT%, O...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,⋯,BLK,BLKA,PF,PFD,+/-,First Player,Second Player,Third Player,Fourth Player,Fifth Player
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
0,ORL,19,21.6,52.3,20.4,42.3,48.2,4.3,10.9,⋯,2.2,2.4,8.0,6.9,0.8,E.Fournier,A.Gordon,E.Payton,T.Ross,N.Vucevic
1,MIN,41,21.5,46.3,17.6,37.2,47.4,3.2,9.1,⋯,2.0,2.2,7.8,8.9,-0.9,G.Dieng,Z.LaVine,R.Rubio,K.Towns,A.Wiggins
2,WAS,69,19.5,45.4,17.7,35.9,49.4,4.1,10.1,⋯,1.4,2.0,7.1,7.1,3.4,B.Beal,M.Gortat,M.Morris,O.Porter,J.Wall
3,LAC,46,18.9,44.0,16.2,31.6,51.3,3.8,8.5,⋯,2.0,1.1,6.2,9.3,5.4,B.Griffin,D.Jordan,L.Mbah,C.Paul,J.Redick
4,CLE,27,16.8,40.9,15.3,31.1,49.0,5.0,11.5,⋯,1.4,1.4,5.6,7.1,2.2,K.Irving,L.James,K.Love,J.Smith,T.Thompson
5,CHA,47,16.7,36.2,13.3,29.4,45.1,3.3,9.3,⋯,1.6,2.2,4.6,6.4,1.7,N.Batum,M.Kidd-Gilchrist,K.Walker,M.Williams,C.Zeller
6,PHX,7,16.4,38.3,14.4,29.9,48.3,2.7,7.9,⋯,0.7,1.9,8.4,6.9,0.1,E.Bledsoe,D.Booker,T.Chandler,J.Dudley,T.Warren
7,MIN,38,15.7,34.8,13.3,27.0,49.5,2.2,5.8,⋯,1.4,1.5,5.3,6.2,0.6,G.Dieng,R.Rubio,B.Rush,K.Towns,A.Wiggins
8,CLE,12,15.1,40.2,14.5,27.7,52.4,5.2,10.6,⋯,1.5,1.4,3.5,6.4,9.8,K.Irving,L.James,D.Liggins,K.Love,T.Thompson
